In [5]:
# IMPORTS
# import sys
# sys.path.insert(0, '../Analysis')
import helpers as h
import empatica_helpers as eh
import inquisit_helpers as ih
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

# ML IMPORTS
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

reload(h), reload(eh), reload(ih)

# GLOBAL SETTINGS
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:.2f}'.format
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('seaborn-v0_8-notebook') # plt.style.use('ggplot'); print(plt.style.available)
pd.set_option('display.max_columns', None)

sr = 32
wl = 24 # Window length in seconds

# FULL PIPELINE
# e_raw, _ = eh.load_empatica(data_folder='input/empatica/', useIBI=False, save=True, plotTrimmings=False, desired_sampling_rate=sr)
# i_raw = ih.load_inquisit(data_folder='input/inquisit/', save=True)
# ei_raw = h.combine_empatica_and_inquisit(e_raw, i_raw, save=True, sr=sr)
# ei_prep = h.clean_and_filter(save=True, normalise=None, sr=sr, window_length=wl)
# X, y, p = h.prepare_for_vae(sr=sr, wl=wl, filepath="output/ei_prep.csv", save=True, normalise=False) # Normalisation now happens later in the process. Normalise = False applies the standard scaler to the data.

X_train, X_val, X_test, y_train, y_val, y_test, p_train, p_val, p_test = h.prepare_train_val_test_sets(filenames=['output/dl_X_wl24_sr32.pkl', 'output/dl_y_wl24_sr32.pkl', 'output/dl_p_wl24_sr32.pkl'])

# Concatenate X_train and X_val to create a new X_train, as well as p_train and p_val to create a new p_train, and y_train and y_val to create a new y_train
X_train_raw = np.concatenate((X_train, X_val), axis=0)
p_train = np.concatenate((p_train, p_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)

# BACK TO INPUTS FOR ML MODELS
X_test = h.prepare_for_ml(X=X_test, y=y_test, wl=24)

# Initialise dicts
space_dict = {} # Store the search space for each classifier
model_dict = {
    'xgb': XGBClassifier,
    'glm': LogisticRegression,
    'rf': RandomForestClassifier,
    'svm': SVC
}

Train size:  80.23809523809524
Val size:  8.333333333333332
Test size:  11.428571428571429
Size: : (1011, 768, 6)


In [6]:
# Print nans in X_train (samples, features)
X_train = h.prepare_for_ml(X_train_raw, wl=24, y=y_train)
print(f"X_train: {X_train.shape}")
print(f"X_train nans:\n{np.isnan(X_train).sum()}")

X_train: (1116, 36)
X_train nans:
temp_mean          0
temp_std           0
temp_min           0
temp_max           0
temp_skew          0
temp_kurt          0
bvp_mean           0
bvp_std            0
bvp_min            0
bvp_max            0
bvp_skew           0
bvp_kurt           0
hr_mean            0
hr_std             0
hr_min             0
hr_max             0
hr_skew            0
hr_kurt            0
body_acc_mean      0
body_acc_std       0
body_acc_min       0
body_acc_max       0
body_acc_skew      0
body_acc_kurt      0
eda_tonic_mean     0
eda_tonic_std      0
eda_tonic_min      0
eda_tonic_max      0
eda_tonic_skew     0
eda_tonic_kurt     0
eda_phasic_mean    0
eda_phasic_std     0
eda_phasic_min     0
eda_phasic_max     0
eda_phasic_skew    0
eda_phasic_kurt    0
dtype: int64


## Defining hyperparameter spaces

In [7]:
reload(h)

# ------------ XGBoost ------------

# Define the hyperparameter space
counts = np.unique(y_train, return_counts=True)[1]
scale_pos_weight = counts[0] / counts[1] # Recommended by: https://webcache.googleusercontent.com/search?q=cache:https://towardsdatascience.com/a-guide-to-xgboost-hyperparameters-87980c7f44a9&sca_esv=254eb9c569a53dbc&strip=1&vwsrc=0
# Default recommendations: https://bradleyboehmke.github.io/xgboost_databricks_tuning/tutorial_docs/xgboost_hyperopt.html

xgb_space = {
    'window_size': hp.choice('window_size', range(8, 24, 2)),
    'objective':'binary:logistic',
    'max_depth': hp.choice('max_depth', np.arange(2, 11, dtype=int)),
    'min_child_weight': hp.uniform('min_child_weight', 0.1, 15),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(1)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.5, 1),
    'n_estimators': hp.choice('n_estimators', range(50, 5000)),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_log', np.log(1), np.log(1000))]),
    'reg_lambda': hp.choice('reg_lambda', [0, hp.loguniform('reg_lambda_log', np.log(1), np.log(1000))]),
    'reg_alpha': hp.choice('reg_alpha', [0, hp.loguniform('reg_alpha_log', np.log(1), np.log(1000))]),
    'scale_pos_weight': scale_pos_weight
}

space_dict['xgb'] = xgb_space

# ------------ GLM ------------

# Define the hyperparameter space
glm_space = {
    'window_size': hp.choice('window_size', range(8, 24, 2)),
    'C': hp.loguniform('C', np.log(0.001), np.log(1000)),
    'penalty': hp.choice('penalty', ['l1', 'l2']),
    'solver': hp.choice('solver', ['liblinear', 'saga']), # Only solvers that support both L1 and L2 penalties
    'class_weight' : 'balanced',
    'max_iter': 10000
}

space_dict['glm'] = glm_space

# ------------ Random Forest ------------

# Source: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
rf_space = {
    'window_size': hp.choice('window_size', range(8, 24, 2)),
    'n_estimators': hp.choice('n_estimators', range(2, 200)),
    'max_depth': hp.choice('max_depth', np.arange(2, 101, dtype=int)),
    'max_features': hp.choice('max_features', ['log2', 'sqrt', None]),
    'min_samples_split': hp.choice('min_samples_split', np.arange(2, 10, dtype=int)),
    'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(1, 5, dtype=int)),
    'class_weight' : 'balanced'
}

space_dict['rf'] = rf_space

# ------------ SVM ------------

# Define the hyperparameter space
svm_space = hp.choice('model_type', [
    {
        'window_size': hp.choice('window_size_linear', range(8, 24, 2)),
        'C': hp.loguniform('C_linear', np.log(0.01), np.log(10)),  # Lower range for C
        'kernel': 'linear',
        'class_weight' : 'balanced'
    },
    {
        'window_size': hp.choice('window_size_rbf', range(8, 24, 2)),
        'C': hp.loguniform('C_rbf', np.log(0.01), np.log(10)),
        'kernel': 'rbf',
        'gamma': hp.loguniform('gamma_rbf', np.log(0.001), np.log(1)),  # Lower range for gamma
        'class_weight' : 'balanced'
    }
])

space_dict['svm'] = svm_space

print(space_dict)

{'xgb': {'window_size': <hyperopt.pyll.base.Apply object at 0x308220590>, 'objective': 'binary:logistic', 'max_depth': <hyperopt.pyll.base.Apply object at 0x3082229d0>, 'min_child_weight': <hyperopt.pyll.base.Apply object at 0x17ec97490>, 'learning_rate': <hyperopt.pyll.base.Apply object at 0x30821da50>, 'subsample': <hyperopt.pyll.base.Apply object at 0x30821ddd0>, 'colsample_bytree': <hyperopt.pyll.base.Apply object at 0x30821e3d0>, 'colsample_bylevel': <hyperopt.pyll.base.Apply object at 0x30821f210>, 'colsample_bynode': <hyperopt.pyll.base.Apply object at 0x30821c550>, 'n_estimators': <hyperopt.pyll.base.Apply object at 0x30ab1c050>, 'gamma': <hyperopt.pyll.base.Apply object at 0x30ab1c990>, 'reg_lambda': <hyperopt.pyll.base.Apply object at 0x30ab1d290>, 'reg_alpha': <hyperopt.pyll.base.Apply object at 0x30ab1db90>, 'scale_pos_weight': 2.024390243902439}, 'glm': {'window_size': <hyperopt.pyll.base.Apply object at 0x30ab1e510>, 'C': <hyperopt.pyll.base.Apply object at 0x30ab1e9d0>, 

## Hyperparameter optimisation with cross-validation

In [8]:
from sklearn.metrics import average_precision_score
import pickle
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
reload(h)

def inject_synthetic_data(X_train_fold, y_train_fold, variational_autoencoder, fraction_synthetic, seed, rebalance):
    np.random.seed(seed)

    # Calculate the number of synthetic samples to generate
    num_synthetic_samples = int(len(X_train_fold) * fraction_synthetic)
    new_total = len(X_train_fold) + num_synthetic_samples

    # If rebalance is True, rebalance the class distribution towards the minority class
    if rebalance:
        # Calculate the number of samples in each class
        num_neg = np.sum(y_train_fold == 0)
        num_pos = np.sum(y_train_fold == 1)

        add_to_neg = (new_total // 2) - num_neg if ((new_total // 2) - num_neg) > 0 else 0
        add_to_pos = (new_total // 2) - num_pos if ((new_total // 2) - num_pos) > 0 else 0

        # Generate synthetic samples for each class
        synthetic_samples_neg = np.random.normal(size=(add_to_neg, X_train_fold.shape[1]))
        synthetic_samples_pos = np.random.normal(size=(add_to_pos, X_train_fold.shape[1]))
        # synthetic_samples_neg = variational_autoencoder.generate_samples(add_to_neg, condition=0)
        # synthetic_samples_pos = variational_autoencoder.generate_samples(add_to_pos, condition=1)

        # Combine the synthetic samples
        synthetic_samples = np.concatenate([synthetic_samples_neg, synthetic_samples_pos])
        synthetic_labels = np.array([0] * add_to_neg + [1] * add_to_pos)
    else:
        # Generate synthetic samples for each class
        synthetic_samples_neg = vae.generate_samples(num_synthetic_samples // 2, condition=0)
        synthetic_samples_pos = vae.generate_samples(num_synthetic_samples // 2, condition=1)

        # Combine the synthetic samples
        synthetic_samples = np.concatenate([synthetic_samples_neg, synthetic_samples_pos])
        synthetic_labels = np.array([0] * len(synthetic_samples_neg) + [1] * len(synthetic_samples_pos))
    
    # Inject the synthetic samples into the training fold
    X_train_fold = np.concatenate([X_train_fold, synthetic_samples])
    y_train_fold = np.concatenate([y_train_fold, synthetic_labels])

    return X_train_fold, y_train_fold

def optimise_model(model, space, max_evals=100):
    def objective(params):
        # Prepare data
        window_size = params.pop('window_size')        
        X_train = h.prepare_for_ml(X=X_train_raw, y=y_train, wl=window_size) # p_train, y_train are already defined
        
        # Create folds
        folds = h.create_folds(X_train, y_train, groups=p_train, n_folds=5) # Exhaustive would be 12 folds
        # Train and evaluate the model using cross-validation
        clf = model(**params, random_state=42)
        scores = []

        for train_index, test_index in folds:
            X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
            y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
            X_train_fold, y_train_fold = inject_synthetic_data(X_train_fold, y_train_fold, variational_autoencoder=vae, window_size=window_size, fraction_synthetic=0.5, seed=42, rebalance=True)
            clf.fit(X_train_fold, y_train_fold)
            y_pred = clf.predict(X_test_fold)
            score = average_precision_score(y_test_fold, y_pred)
            scores.append(score)
        score = np.mean(scores)  # Use the average score

        return {'loss': -score, 'status': STATUS_OK, 'params': params}

    # Perform the optimisation
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_evals)
    best_params = space_eval(space, best)
    return best_params

def optimise_models(space_dict, model_dict, max_evals=100):
    best_params_dict = {}
    for key, space in space_dict.items():
        best_params = optimise_model(model_dict[key], space, max_evals=max_evals)
        print(f"Best parameters for {key}: {best_params}")
        best_params_dict[key] = best_params
    return best_params_dict

best_params_dict = optimise_models(space_dict, model_dict, max_evals=100)

# Save the best parameters
# with open('output/ml_best_params_cv_nt.pkl', 'wb') as f:
#     pickle.dump(best_params_dict, f)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: name 'vae' is not defined



  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


NameError: name 'vae' is not defined

### Optimisation w/ synthetic data injection w/ returning all scores for statistical testing

In [13]:
from sklearn.metrics import average_precision_score
import pickle
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
reload(h)

def optimise_model(model, space, max_evals=100):
    def objective(params):
        # Prepare data
        window_size = params.pop('window_size')        
        X_train = h.prepare_for_ml(X=X_train_raw, y=y_train, wl=window_size) # p_train, y_train are already defined
        
        # Create folds
        folds = h.create_folds(X_train, y_train, groups=p_train, n_folds=5) # Exhaustive would be 12 folds
        # Train and evaluate the model using cross-validation
        clf = model(**params, random_state=42)
        scores = []

        for train_index, test_index in folds:
            X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
            y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
            X_train_fold, y_train_fold = h.inject_synthetic_data(X_train_fold, y_train_fold, fraction_synthetic=0.5, seed=42, rebalance=True)
            clf.fit(X_train_fold, y_train_fold)
            y_pred = clf.predict(X_test_fold)
            score = average_precision_score(y_test_fold, y_pred)
            scores.append(score)

        return {'loss': -np.mean(scores), 'status': STATUS_OK, 'params': params, 'scores': scores}

    # Perform the optimisation
    trials = Trials()
    best = fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    best_params = space_eval(space, best)
    best_scores = trials.best_trial['result']['scores']
    return best_params, best_scores

def optimise_models(space_dict, model_dict, max_evals=100):
    best_params_dict = {}
    best_scores_dict = {}
    for key, space in space_dict.items():
        best_params, best_scores = optimise_model(model_dict[key], space, max_evals=max_evals)
        print(f"Best parameters for {key}: {best_params}")
        print(f"Best scores for {key}: {best_scores}")
        best_params_dict[key] = best_params
        best_scores_dict[key] = best_scores
    return best_params_dict, best_scores_dict

best_params_dict, best_scores_dict = optimise_models(space_dict, model_dict, max_evals=100)

# Save the best parameters
# with open('output/ml_best_params_cv_nt.pkl', 'wb') as f:
#     pickle.dump(best_params_dict, f)

  2%|▏         | 2/100 [00:08<07:54,  4.84s/trial, best loss: -0.3501198225656577] 